In [1]:
# Python CODE
import numpy as np
# DO NOT FORGET TO SPECIFY THE SAME SEED
np.random.seed ( 12345 )
def initialize(input_dim, hidden_dim,  output_dim, batch_size):
    W1 = np.random.randn ( hidden_dim , input_dim ) * 0.01
    b1 = np.zeros ( ( hidden_dim , ) )
    W2 = np.random.randn ( hidden_dim , hidden_dim ) * 0.01
    b2 = np.zeros ( ( hidden_dim , ) )
    W3 = np.random.randn ( output_dim , hidden_dim ) * 0.01
    b3 = np.zeros ( ( output_dim , ) )
    b1 = np.reshape( b1, ((b1.shape)[0],1)  )
    b2 = np.reshape( b2, ((b2.shape)[0],1)  )
    b3 = np.reshape( b3, ((b3.shape)[0],1)  )
    # list of all  network parameters
    parameters = [W1, b1 , W2, b2 , W3, b3 ]
    # minibatch of input instances
    x = np.random.rand ( input_dim , batch_size )
    # ground truths
    y = np.random.randn ( output_dim , batch_size )
    return parameters , x , y
# initialize parameters , inputs and targets
parameters, X, Y = initialize ( 3 , 4 , 2 , 5 )

## question 01
forward pass and loss function

In [2]:
def sigmoid(inX):  
    return 1.0/(1+np.exp(-inX))     
def f_NN(parameters, x):
    [W1, b1, W2, b2, W3, b3] = parameters 
    o1 = np.dot(W1,x)  + b1
    H1 = sigmoid( o1  ) #
    o2 = np.dot(W2,H1) + b2
    H2 = sigmoid( o2  ) #
    o3 = np.dot(W3,H2) + b3
    forward = [o1, H1, o2, H2, o3]
    return forward
def loss( parameters, x, y):
    forward = f_NN(parameters, x)
    [o1, H1, o2, H2, o3] = forward
    guess_y = o3
    L = 0
    steps = y.shape
    steps = steps[1]
    for i in range( steps  ):
        err = guess_y[:,i] - y[:,i]
        L += np.linalg.norm( err ) 
    return L/(2*steps) , forward
    

## question 2
backpropagation  
dx_dx means partial derivation
Xi_ij means $i_{th}$ layer, ij element \\
for vector xj
$$ dL/dxx \\   
   .../do3 = (o3 -y)^T\\
   o = Wx+b \quad given \quad dL/do \\
   dL/db = dL/do\\
       dL/dW = x \times dL/do \quad (outer)\\
       dL/dx = dL/do \times W  \\
   h = sig(o) \quad given\quad dL/dh \\
   dL/do = dL/dh \times (1-h) \times h \quad (np.multiply)
$$

In [3]:
Loss, forward = loss( parameters, X, Y)
[O1, H1, O2, H2, O3] = forward
[W1, b1, W2, b2, W3, b3] = parameters
steps = Y.shape
steps = steps[1] 
detW1 = {}
detb1 = {}
detW2 = {}
detb2 = {}
detW3 = {}
detb3 = {}
for i in range (steps):
    [o1,h1,o2,h2,o3,x,y] = [ O1[:,i], H1[:,i], O2[:,i], H2[:,i], O3[:,i], X[:,i], Y[:,i] ]
    
    dL_do3 = o3 - y
    
    dL_db3 = dL_do3
    dL_dW3 = np.outer(h2, dL_do3)
    dL_dh2 = np.dot(dL_do3, W3)
    dL_do2 = dL_dh2 * (1-h2) * h2
    
    dL_db2 = dL_do2
    dL_dW2 = np.outer(h1, dL_do2)
    dL_dh1 = np.dot(dL_do2, W2)
    dL_do1 = dL_dh1 * (1-h1) * h1
    
    dL_db1 = dL_do1
    dL_dW1 = np.outer(x, dL_do1)    
    if i == 0:
        detW1 = dL_dW1
        detb1 = dL_db1
        detW2 = dL_dW2
        detb2 = dL_db2
        detW3 = dL_dW3
        detb3 = dL_db3              
    else:
        detW1 += dL_dW1
        detb1 += dL_db1
        detW2 += dL_dW2
        detb2 += dL_db2
        detW3 += dL_dW3
        detb3 += dL_db3        
detW1 /= steps
detb1 /= steps
detW2 /= steps
detb2 /= steps
detW3 /= steps
detb3 /= steps
err = [detW1,detb1,detW2,detb2,detW3,detb3]


## Question 3
gradient checking program

In [4]:
def NG_check(parameters, X, Y):
    steps = len(parameters)
    gradient = []
    for index in range(steps):
        size = (parameters[index]).shape
        gradient.append( ( np.zeros(size) ).T  )
        for i, row in enumerate(parameters[index]):
            for j, ele in enumerate(row):
                temp = parameters[index][i][j]
                if temp == 0:
                    parameters[index][i][j] = 0.01
                    lossA, forward = loss( parameters, X, Y)
                    parameters[index][i][j] = 0
                    parameters[index][i][j] = -0.01
                    lossB, forward = loss( parameters, X, Y)
                    parameters[index][i][j] = 0
                    gradient[index][j][i] = (lossA - lossB)/(2*0.01)
                    
                else:
                    delta = temp * 0.01
                    parameters[index][i][j] += delta
                    lossA, forward = loss( parameters, X, Y)
                    parameters[index][i][j] = temp
                    parameters[index][i][j] -= delta
                    lossB, forward = loss( parameters, X, Y)
                    parameters[index][i][j] = temp
                    gradient[index][j][i] = (lossA - lossB)/(2*delta)
    return gradient    
 
gradient = NG_check(parameters, X, Y)  

#### analytical gradient  
the order $\frac{dL}{d..}$  
W1 b1 W2 b2 W3 b3

In [5]:
err

[array([[-3.55164801e-07,  2.02412923e-06,  1.12728984e-05,
          6.96166795e-07],
        [ 4.19035853e-07,  1.85604158e-06,  8.49377219e-06,
          6.27452005e-07],
        [ 3.52847230e-06,  1.01560709e-06, -3.51794858e-06,
          2.95038713e-07]]),
 array([6.68064134e-06, 3.46671844e-06, 1.26685662e-06, 1.08561520e-06]),
 array([[ 3.99898284e-06, -9.83803475e-04,  5.26033041e-04,
          6.09661103e-04],
        [ 4.14561394e-06, -9.89356662e-04,  5.29099114e-04,
          6.13133340e-04],
        [ 3.09285858e-06, -9.86819589e-04,  5.26928985e-04,
          6.11301153e-04],
        [ 1.00056924e-05, -9.82852021e-04,  5.30215253e-04,
          6.10570429e-04]]),
 array([ 6.07123945e-06, -1.96949231e-03,  1.05156448e-03,  1.22000815e-03]),
 array([[-0.29601562, -0.00063616],
        [-0.29315414, -0.00062495],
        [-0.29558964, -0.00062947],
        [-0.29427093, -0.00063094]]),
 array([-0.58798803, -0.00125797])]

#### numerical gradient
the order is the same as before

In [6]:
gradient

[array([[4.19325363e-07, 1.46553834e-06, 6.48765782e-06, 4.94140582e-07],
        [5.05338732e-08, 1.09282369e-06, 5.48659417e-06, 3.72307283e-07],
        [1.40800583e-06, 7.12534872e-07, 1.73978681e-07, 2.22622129e-07]]),
 array([[2.26522746e-06, 2.09244221e-06, 5.13813055e-06, 6.81121454e-07]]),
 array([[ 0.00023584, -0.0003835 ,  0.0003879 ,  0.00029608],
        [ 0.00023727, -0.00038574,  0.0003902 ,  0.00029782],
        [ 0.00023602, -0.00038496,  0.00038881,  0.00029703],
        [ 0.00023889, -0.00038382,  0.00039045,  0.00029704]]),
 array([[ 0.000471  , -0.00076802,  0.00077581,  0.00059262]]),
 array([[-0.1705999 , -0.04008685],
        [-0.16895066, -0.03969746],
        [-0.17035409, -0.04002706],
        [-0.16959428, -0.03985005]]),
 array([[-0.33864654, -0.07937331]])]